# Descrição do Projeto

Seguem algumas instruções para realizar os experimentos:

-- selecionar um banco de dados do OpenML (https://www.openml.org/)
     -- o banco de dados de classificação deve ter: > 3 classes, >10 atributos e >1000 instâncias
     -- cada dupla deve informar qual banco de dados usará com antecedência
-- 10-fold cross-validation
-- avaliar máquinas de aprendizagem: kNN, DT, MLP, NB, SVM, … 
-- métricas (area under the ROC curve, f1-score, accuracy, matriz de confusão média)
-- para ajustar os parâmetros das máquinas de aprendizagem usar grid-search ou escolher os parâmetros e justificar

O relatório final deve conter pelo menos as seguintes informações:
  1. Introdução
  2. Algoritmos de Aprendizagem de Máquina
  3. Experimentos
    3.1. Banco de dados
    3.2. Métricas
    3.3. Resultados
    4.Conclusões
  
  Referências

Grupo com no máximo dois integrantes.

# Importações

In [34]:
import pandas as pd
import numpy as np
from scipy.io import arff
import matplotlib.pyplot as plt

# Classes do modelo de aprendizado
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree

# Funções de avaliação dos modelos
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Funções

# Scripts

## Obtenção dos dados

Dados obtidos de: [gas-drift](https://www.openml.org/search?type=data&sort=runs&id=1476&status=active)

In [13]:
# Ler o arquivo .arff
arff_data = arff.loadarff(open("../data/raw/phpbL6t4U.arff", 'r', encoding='utf-8'))

# Converter os dados em um DataFrame do Pandas
df = pd.DataFrame(arff_data[0])

df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V120,V121,V122,V123,V124,V125,V126,V127,V128,Class
0,12285.6582,4.076635,4.842317,7.509393,10.822436,-1.312657,-1.853717,-6.924985,11800.9233,4.483500,...,-3.440387,1784.5324,1.907000,1.729200,4.881194,8.623828,-0.314110,-0.661556,-3.521663,b'4'
1,-35.6889,0.993944,0.166099,0.489363,3.484663,-0.130298,-0.528364,-3.735347,266.4145,1.053988,...,-4.482534,904.9898,1.433707,1.068069,2.532958,5.369720,-0.183779,-0.534087,-4.635975,b'3'
2,63927.2217,14.956941,19.971376,29.188512,33.291320,-10.433776,-16.062245,-49.490143,57405.8483,15.613843,...,-25.150090,14585.7879,8.189021,6.099452,12.127991,15.709651,-3.887082,-6.731473,-19.326895,b'4'
3,2992.9019,1.380553,0.808910,1.288259,4.660135,-0.755903,-1.120470,-4.075213,4301.4033,1.652701,...,-7.215792,6044.5554,3.488295,2.662288,5.938297,8.544508,-1.567322,-2.701235,-6.472439,b'3'
4,57524.7812,11.912566,14.631496,19.809240,23.715868,-9.084750,-11.770585,-39.234003,50051.0703,11.732548,...,-10.248794,10580.1006,5.752675,3.880740,8.545897,11.831716,-2.655521,-4.312744,-8.510591,b'4'


In [15]:
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128
count,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,...,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000,13910.000000
mean,50435.066174,6.638156,12.936688,18.743953,26.890695,-9.158655,-14.402383,-59.927598,57340.104585,6.648033,...,-9.601927,-19.136500,19688.565373,6.072066,7.138634,14.929364,19.090980,-4.901016,-8.167792,-16.089791
std,69844.785952,13.486391,17.610061,24.899450,38.107685,12.729206,21.304606,131.017675,64045.265134,15.585780,...,9.220031,26.516679,14281.652395,4.642192,5.248573,12.437311,14.391810,4.195360,7.637701,20.958479
min,-16757.598600,0.088287,0.000100,0.000100,0.000100,-131.332873,-227.627758,-1664.735576,-16119.460900,0.185164,...,-76.069200,-482.278033,-8297.548800,0.712112,0.003238,0.011488,0.118849,-30.205911,-58.844076,-410.152297
25%,6694.725950,2.284843,1.633350,2.386836,4.967988,-11.587169,-17.292559,-48.492764,13287.301875,2.776693,...,-13.212575,-22.363498,8837.838750,3.007381,3.059178,5.407551,8.039227,-6.789599,-11.162406,-18.938690
50%,19364.439350,3.871227,4.977123,7.250892,11.680725,-3.338700,-4.956917,-14.040088,37764.263200,4.734586,...,-7.338850,-13.527887,16313.967300,4.973783,5.809107,11.325214,14.560676,-3.881763,-6.305962,-11.747499
75%,63104.837125,8.400619,17.189166,26.411109,34.843226,-1.126897,-1.670327,-5.212213,70300.782575,8.608522,...,-3.260080,-7.358031,27009.592425,7.389566,10.222169,21.207572,26.547437,-1.804032,-2.874532,-6.429690
max,670687.347700,1339.879283,167.079751,226.619457,993.605306,-0.006941,22.201589,115.273147,502202.812500,1672.363221,...,9.270956,11.516418,96706.792700,45.574835,32.203601,297.225880,195.242555,-0.003817,6.851792,8.357968


In [14]:
# Verificando a quantidade de registros para cada classe
df['Class'].value_counts()

Class
b'5'    3009
b'2'    2926
b'1'    2565
b'4'    1936
b'6'    1833
b'3'    1641
Name: count, dtype: int64

As restrições dadas para o projeto são respeitadas:

* >3 classes: 6
* >10 atributos: 128
* >1000 instâncias: 13.900

## Máquinas de Aprendizagem

### Divisão

In [23]:
df['Class2'] = pd.factorize(df['Class'])[0]

In [25]:
df['Class2'].value_counts()

Class2
5    3009
3    2926
4    2565
0    1936
2    1833
1    1641
Name: count, dtype: int64

In [26]:
#Vamos usar somente duas features SepalLengthCm e SepalWidthCm
X = df.drop(columns=["Class", "Class2"])
y = df["Class2"]

#Separando o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [27]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128
0,12285.6582,4.076635,4.842317,7.509393,10.822436,-1.312657,-1.853717,-6.924985,11800.9233,4.483500,...,-0.747592,-3.440387,1784.5324,1.907000,1.729200,4.881194,8.623828,-0.314110,-0.661556,-3.521663
1,-35.6889,0.993944,0.166099,0.489363,3.484663,-0.130298,-0.528364,-3.735347,266.4145,1.053988,...,-0.622342,-4.482534,904.9898,1.433707,1.068069,2.532958,5.369720,-0.183779,-0.534087,-4.635975
2,63927.2217,14.956941,19.971376,29.188512,33.291320,-10.433776,-16.062245,-49.490143,57405.8483,15.613843,...,-7.762551,-25.150090,14585.7879,8.189021,6.099452,12.127991,15.709651,-3.887082,-6.731473,-19.326895
3,2992.9019,1.380553,0.808910,1.288259,4.660135,-0.755903,-1.120470,-4.075213,4301.4033,1.652701,...,-3.244969,-7.215792,6044.5554,3.488295,2.662288,5.938297,8.544508,-1.567322,-2.701235,-6.472439
4,57524.7812,11.912566,14.631496,19.809240,23.715868,-9.084750,-11.770585,-39.234003,50051.0703,11.732548,...,-5.093413,-10.248794,10580.1006,5.752675,3.880740,8.545897,11.831716,-2.655521,-4.312744,-8.510591


Necessário normalizar as variáveis

In [28]:
y.head()

0    0
1    1
2    0
3    1
4    0
Name: Class2, dtype: int64

### KNN

In [29]:
# vamos criar um classificador kNN com k=5
model = KNeighborsClassifier()
model.fit(X_train, y_train)

# e ver a sua performance no dataset de teste
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       555
           1       0.99      0.96      0.98       481
           2       0.97      0.99      0.98       557
           3       0.98      0.99      0.99       869
           4       0.99      0.97      0.98       779
           5       0.99      0.98      0.98       932

    accuracy                           0.98      4173
   macro avg       0.98      0.98      0.98      4173
weighted avg       0.98      0.98      0.98      4173



### Árvore de Decisão

In [31]:
#scikit-learn usa uma versão otimizada do algoritmo CART (similar ao C4.5)
model = DecisionTreeClassifier(max_depth=3)
#treinando o modelo
model.fit(X_train, y_train)

#predição
y_pred = model.predict(X_test)

#Resultados do classificador
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.32      0.81      0.46       555
           1       0.92      0.73      0.81       481
           2       0.00      0.00      0.00       557
           3       0.70      0.80      0.75       869
           4       0.79      0.80      0.80       779
           5       0.98      0.65      0.78       932

    accuracy                           0.65      4173
   macro avg       0.62      0.63      0.60      4173
weighted avg       0.66      0.65      0.63      4173



C:\Users\mauri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mauri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mauri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classificati

In [35]:
plt.figure(figsize=(100,100))

# Retorna uma lista contendo os dados para as caixas de anotação que compõem a árvore
tree = plot_tree(model, filled=True)